# Required Capacity

Calculate the required capacity for a future file system based on the _daily growth in used capacity_ observed on an existing file system.

In [ ]:
%matplotlib inline

In [ ]:
import os
import glob
import datetime
import warnings

import numpy
import scipy
import matplotlib
import matplotlib.pyplot

import pandas
import fsanalysis.peakdetect as peakdetect


In [ ]:
matplotlib.rcParams['font.size'] = 16

In [ ]:
START_TIME = datetime.datetime(2017, 4, 1)
END_TIME = datetime.datetime(2019, 3, 31)

CSCRATCH_KIBS = 29763608416864 # from df
CSCRATCH_GIBS = CSCRATCH_KIBS / 1024.0 / 1024.0
CSCRATCH_TIBS = CSCRATCH_GIBS / 1024.0

Load file system fullness data from CSV or from raw input sources

In [ ]:
CACHE_FILE = 'datasets/cscratch_fullness-2017-02-14_2019-03-31.csv'
print("Loading from", CACHE_FILE)
cscratch_lfs = pandas.read_csv(CACHE_FILE, index_col=0)
print("Loaded %d days (out of %d expected)" % (len(cscratch_lfs), (END_TIME - START_TIME).days))

# Add a new date column with the epoch timestamp so we can calculate daily growth rates
cscratch_lfs['date'] = [int(datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").strftime("%s")) for x in cscratch_lfs.index]
cscratch_lfs = cscratch_lfs[(cscratch_lfs['date'] >= START_TIME.timestamp()) & (cscratch_lfs['date'] <= END_TIME.timestamp())]
print("%d actual days loaded" % len(cscratch_lfs))

In [ ]:
def print_summary(pct_per_sec):
    pct_per_day = pct_per_sec * 86400
    gib_per_day = (pct_per_day / 100.0 * CSCRATCH_GIBS)
    dram_per_day = (gib_per_day / CORI_DRAM_GIB)
    print("File system fills at %.2f%% per day" % pct_per_day)
    print("                     %.2f TiB/day" % (gib_per_day / 1024.0))
    print("                     %.2f%% of DRAM per day" % (dram_per_day * 100.0))
    print("By extrapolation, %d days to fill" % datetime.timedelta(seconds=(100.0 / pct_per_sec)).days)
    return {
                'pct_per_day': pct_per_day,
                'gib_per_day': gib_per_day,
                'dram_per_day': dram_per_day,
           }

In [ ]:
def pibs_to_pbs(pibs):
    return pibs * 2**50 / 10**(5*3)

## Basic visualization of the data

We can calculate the dates on which purges were initiated from the purge logs maintained by Kirill.

In [ ]:
# Figure out the dates on which purges were initiated
purge_dates = []
for logfile in glob.glob('/global/project/projectdirs/m888/glock/cpurge01/purge.cscratch1.*.log'):
    purge_dates.append(int(datetime.datetime.strptime(os.path.basename(logfile).split('.')[2], "%Y%m%d").strftime("%s")))
purge_dates.sort()

Plot the file system fullness along with purge start dates to see how file system capacity responded to the various purges that have been run on cscratch.

In [ ]:
fig, ax = matplotlib.pyplot.subplots(figsize=(12,4))

ax.plot([datetime.datetime.fromtimestamp(xx) for xx in cscratch_lfs['date'].values],
        (cscratch_lfs['ost_avg_full_pct']).values,
       label="cscratch")
#ax.plot(sma7)
#ax.plot(sma28)

ax.set_ylim(0, 100)
ymin, ymax = ax.get_ylim()

# Plot purge dates
for purgedate in purge_dates:
    if purgedate < cscratch_lfs['date'].max():
        ax.plot([datetime.datetime.fromtimestamp(purgedate),
                 datetime.datetime.fromtimestamp(purgedate)],
                [ymin, ymax],
                '--',
                color='red')

ax.legend()

In [ ]:
daily_fill_pct = cscratch_lfs['ost_avg_full_pct'].iloc[1:].values - cscratch_lfs['ost_avg_full_pct'].iloc[0:-1].values
daily_fill_pct = daily_fill_pct[daily_fill_pct >= 0.0]

## Distribution of daily file system growth rates

In [ ]:
daily_fill_tibs = daily_fill_pct / 100.0 * CSCRATCH_TIBS
daily_fill_tbs = numpy.vectorize(lambda x: (pibs_to_pbs(x/1024.0)*1000.0))(daily_fill_tibs)

print("%.2f TiB mean daily growth" % (daily_fill_tibs).mean())
print("%.2f TiB min daily growth" % (daily_fill_tibs).min())
print("%.2f TiB 25th daily growth" % scipy.percentile(daily_fill_tibs, 25))
print("%.2f TiB median daily growth" % scipy.percentile(daily_fill_tibs, 50))
print("%.2f TiB 75th daily growth" % scipy.percentile(daily_fill_tibs, 75))
print("%.2f TiB max daily growth" % (daily_fill_tibs).max())
print()
print("%.2f TB mean daily growth" % (daily_fill_tbs).mean())
print("%.2f TB min daily growth" % (daily_fill_tbs).min())
print("%.2f TB 25th daily growth" % scipy.percentile(daily_fill_tbs, 25))
print("%.2f TB median daily growth" % scipy.percentile(daily_fill_tbs, 50))
print("%.2f TB 75th daily growth" % scipy.percentile(daily_fill_tbs, 75))
print("%.2f TB max daily growth" % (daily_fill_tbs).max())

In [ ]:
bin_width = 50.0
max_bin = 1000.0
fig, ax = matplotlib.pyplot.subplots(figsize=(8, 4))
ax.hist(daily_fill_pct / 100.0 * CSCRATCH_TIBS,
        edgecolor='black',
        width=bin_width * 0.90,
        color='C0',
        bins=numpy.arange(0, max_bin, bin_width))
ax.set_ylabel("Number of days")
ax.set_xlabel("Daily growth (TiB/day)")
ax.set_ylim(-3, None)
ax.set_xticks(numpy.arange(0, max_bin, bin_width))
ax.set_xticklabels(["%d" % x if (x % int(2*bin_width) == 0) else "" for x in numpy.arange(0, max_bin, bin_width)],
                   rotation=30,
                   ha="center")

ax.yaxis.grid()
ax.set_axisbelow(True)
caption = "Cori scratch (%.1f PB)\n%s - %s" % (
    pibs_to_pbs(CSCRATCH_TIBS / 1024.0),
    START_TIME.strftime("%b %-d, %Y"),
    END_TIME.strftime("%b %-d, %Y"))
ax.text(0.98, 0.84, caption, fontsize='medium',
        ha='right', transform=ax.transAxes, backgroundcolor='#FFFFFF99')

In [ ]:
output_file = 'cscratch_growth_rate_%s-%s.pdf' % (START_TIME.strftime("%Y%m%d"), END_TIME.strftime("%Y%m%d"))
fig.savefig(output_file, dpi=200, bbox_inches='tight', transparent=True)
print("Wrote output to", output_file)

## Calculating Required File System Capacity

Here we determine the required file system capacity for a range of input parameters (purge policies, SSIs, etc)

In [ ]:
percentile = 75
purge_fraction = 0.50
purge_interval = 28
rate_multipliers = [3.0, 4.0]

# fill_rate_tibs = scipy.percentile(daily_fill_pct * CSCRATCH_TIBS, percentile) / 100.0
fill_rate_tibs = daily_fill_pct.mean() * CSCRATCH_TIBS / 100.0

def calc_pibs_required(fill_rate_tibs, purge_fraction, purge_interval, rate_multiplier):
    fill_rate_pibs = fill_rate_tibs / 1024.0
    return (rate_multiplier * fill_rate_pibs * purge_interval / purge_fraction)

for ssi in rate_multipliers:
    for purge_interval in (21, 28, 84):
        print("=====" * 20)
        print("Examining purge interval of %d days at SSI = %.1f" % (purge_interval, ssi))
        print("=====" * 20)

        print("The chosen fill rate is %.2f TiB/day" % fill_rate_tibs)
        print("                        %.2f TB/day" % (pibs_to_pbs(fill_rate_tibs/1024.0)*1000.0))
        print("%.1fx increase in throughput on N9 will fill at %.1f TB/day" % (ssi, ssi * fill_rate_tibs))
        print("which would take %d days to fill" % (30000.0 / (ssi * fill_rate_tibs)))

        print()

        print("Assuming we purge %d%% of bytes every %d days," % (purge_fraction * 100.0, purge_interval))

        pibs_required = calc_pibs_required(fill_rate_tibs, purge_fraction, purge_interval, ssi)
        print("PiB required: %.2f" % pibs_required)
        print("PB required: %.2f" % pibs_to_pbs(pibs_required))
        print()